In [1]:
import os, SimpleITK as sitk

root = r"C:\Users\Negar\Desktop\paper_results\Myself\cr_coad_project\data\raw\tcia"

# find a folder that already has some .dcm files (NBIA may still be downloading)
dcm_dir = None
for r, d, f in os.walk(root):
    if any(x.lower().endswith(".dcm") for x in f):
        dcm_dir = r
        break

if dcm_dir is None:
    raise SystemExit("No DICOM files found yet. Let NBIA run a bit longer, then re-run this cell.")

series_ids = sitk.ImageSeriesReader.GetGDCMSeriesIDs(dcm_dir)
print("Found series:", series_ids[:3])

files = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(dcm_dir, series_ids[0])
img = sitk.ReadImage(files)
arr = sitk.GetArrayFromImage(img)   # [z, y, x]
print("shape:", arr.shape, "spacing:", img.GetSpacing())


Found series: ('1.3.6.1.4.1.14519.5.2.1.9203.8273.370971589400256299984427146600',)
shape: (1, 439, 512, 512) spacing: (1.0, 1.0, 1.0, 1.0)
